In [2]:
#######################################################################
# Copyright (C)                                                       #
# 2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm

# world height
WORLD_HEIGHT = 4

# world width
WORLD_WIDTH = 12

# probability for exploration
EPSILON = 0.1

# step size
ALPHA = 0.5

# gamma for Q-Learning and Expected Sarsa
GAMMA = 1

# all possible actions
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3
ACTIONS = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]

# initial state action pair values
START = [3, 0]
GOAL = [3, 11]

def step(state, action):
    i, j = state
    if action == ACTION_UP:
        next_state = [max(i - 1, 0), j]
    elif action == ACTION_LEFT:
        next_state = [i, max(j - 1, 0)]
    elif action == ACTION_RIGHT:
        next_state = [i, min(j + 1, WORLD_WIDTH - 1)]
    elif action == ACTION_DOWN:
        next_state = [min(i + 1, WORLD_HEIGHT - 1), j]
    else:
        assert False

    reward = -1
    if (action == ACTION_DOWN and i == 2 and 1 <= j <= 10) or (
        action == ACTION_RIGHT and state == START):
        reward = -100
        next_state = START

    return next_state, reward

# reward for each action in each state
# actionRewards = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
# actionRewards[:, :, :] = -1.0
# actionRewards[2, 1:11, ACTION_DOWN] = -100.0
# actionRewards[3, 0, ACTION_RIGHT] = -100.0

# set up destinations for each action in each state
# actionDestination = []
# for i in range(0, WORLD_HEIGHT):
#     actionDestination.append([])
#     for j in range(0, WORLD_WIDTH):
#         destinaion = dict()
#         destinaion[ACTION_UP] = [max(i - 1, 0), j]
#         destinaion[ACTION_LEFT] = [i, max(j - 1, 0)]
#         destinaion[ACTION_RIGHT] = [i, min(j + 1, WORLD_WIDTH - 1)]
#         if i == 2 and 1 <= j <= 10:
#             destinaion[ACTION_DOWN] = START
#         else:
#             destinaion[ACTION_DOWN] = [min(i + 1, WORLD_HEIGHT - 1), j]
#         actionDestination[-1].append(destinaion)
# actionDestination[3][0][ACTION_RIGHT] = START

# choose an action based on epsilon greedy algorithm
def choose_action(state, q_value):
    if np.random.binomial(1, EPSILON) == 1:
        return np.random.choice(ACTIONS)
    else:
        values_ = q_value[state[0], state[1], :]
        return np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

# an episode with Sarsa
# @q_value: values for state action pair, will be updated
# @expected: if True, will use expected Sarsa algorithm
# @step_size: step size for updating
# @return: total rewards within this episode
def sarsa(q_value, expected=False, step_size=ALPHA):
    state = START
    action = choose_action(state, q_value)
    rewards = 0.0
    while state != GOAL:
        next_state, reward = step(state, action)
        next_action = choose_action(next_state, q_value)
        rewards += reward
        if not expected:
            target = q_value[next_state[0], next_state[1], next_action]
        else:
            # calculate the expected value of new state
            target = 0.0
            q_next = q_value[next_state[0], next_state[1], :]
            best_actions = np.argwhere(q_next == np.max(q_next))
            for action_ in ACTIONS:
                if action_ in best_actions:
                    target += ((1.0 - EPSILON) / len(best_actions) + EPSILON / len(ACTIONS)) * q_value[next_state[0], next_state[1], action_]
                else:
                    target += EPSILON / len(ACTIONS) * q_value[next_state[0], next_state[1], action_]
        target *= GAMMA
        q_value[state[0], state[1], action] += step_size * (
                reward + target - q_value[state[0], state[1], action])
        state = next_state
        action = next_action
    return rewards

# an episode with Q-Learning
# @q_value: values for state action pair, will be updated
# @step_size: step size for updating
# @return: total rewards within this episode
def q_learning(q_value, step_size=ALPHA):
    state = START
    rewards = 0.0
    while state != GOAL:
        action = choose_action(state, q_value)
        next_state, reward = step(state, action)
        rewards += reward
        # Q-Learning update
        q_value[state[0], state[1], action] += step_size * (
                reward + GAMMA * np.max(q_value[next_state[0], next_state[1], :]) -
                q_value[state[0], state[1], action])
        state = next_state
    return rewards

# print optimal policy
def print_optimal_policy(q_value):
    optimal_policy = []
    for i in range(0, WORLD_HEIGHT):
        optimal_policy.append([])
        for j in range(0, WORLD_WIDTH):
            if [i, j] == GOAL:
                optimal_policy[-1].append('G')
                continue
            bestAction = np.argmax(q_value[i, j, :])
            if bestAction == ACTION_UP:
                optimal_policy[-1].append('U')
            elif bestAction == ACTION_DOWN:
                optimal_policy[-1].append('D')
            elif bestAction == ACTION_LEFT:
                optimal_policy[-1].append('L')
            elif bestAction == ACTION_RIGHT:
                optimal_policy[-1].append('R')
    for row in optimal_policy:
        print(row)

# Use multiple runs instead of a single run and a sliding window
# With a single run I failed to present a smooth curve
# However the optimal policy converges well with a single run
# Sarsa converges to the safe path, while Q-Learning converges to the optimal path
def figure_6_4():
    # episodes of each run
    episodes = 500

    # perform 40 independent runs
    runs = 50

    rewards_sarsa = np.zeros(episodes)
    rewards_q_learning = np.zeros(episodes)
    for r in tqdm(range(runs)):
        q_sarsa = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
        q_q_learning = np.copy(q_sarsa)
        for i in range(0, episodes):
            # cut off the value by -100 to draw the figure more elegantly
            # rewards_sarsa[i] += max(sarsa(q_sarsa), -100)
            # rewards_q_learning[i] += max(q_learning(q_q_learning), -100)
            rewards_sarsa[i] += sarsa(q_sarsa)
            rewards_q_learning[i] += q_learning(q_q_learning)

    # averaging over independt runs
    rewards_sarsa /= runs
    rewards_q_learning /= runs

    # draw reward curves
    plt.plot(rewards_sarsa, label='Sarsa')
    plt.plot(rewards_q_learning, label='Q-Learning')
    plt.xlabel('Episodes')
    plt.ylabel('Sum of rewards during episode')
    plt.ylim([-100, 0])
    plt.legend()

    plt.savefig('figure_6_4.png')
    plt.close()

    # display optimal policy
    print('Sarsa Optimal Policy:')
    print_optimal_policy(q_sarsa)
    print('Q-Learning Optimal Policy:')
    print_optimal_policy(q_q_learning)

# Due to limited capacity of calculation of my machine, I can't complete this experiment
# with 100,000 episodes and 50,000 runs to get the fully averaged performance
# However even I only play for 1,000 episodes and 10 runs, the curves looks still good.
def figure_6_6():
    step_sizes = np.arange(0.1, 1.1, 0.1)
    episodes = 1000
    runs = 10

    ASY_SARSA = 0
    ASY_EXPECTED_SARSA = 1
    ASY_QLEARNING = 2
    INT_SARSA = 3
    INT_EXPECTED_SARSA = 4
    INT_QLEARNING = 5
    methods = range(0, 6)

    performace = np.zeros((6, len(step_sizes)))
    for run in range(runs):
        for ind, step_size in tqdm(list(zip(range(0, len(step_sizes)), step_sizes))):
            q_sarsa = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
            q_expected_sarsa = np.copy(q_sarsa)
            q_q_learning = np.copy(q_sarsa)
            for ep in range(episodes):
                sarsa_reward = sarsa(q_sarsa, expected=False, step_size=step_size)
                print(sarsa_reward,ind)
                expected_sarsa_reward = sarsa(q_expected_sarsa, expected=True, step_size=step_size)
                q_learning_reward = q_learning(q_q_learning, step_size=step_size)
                performace[ASY_SARSA, ind] += sarsa_reward
                performace[ASY_EXPECTED_SARSA, ind] += expected_sarsa_reward
                performace[ASY_QLEARNING, ind] += q_learning_reward

                if ep < 100:
                    performace[INT_SARSA, ind] += sarsa_reward
                    performace[INT_EXPECTED_SARSA, ind] += expected_sarsa_reward
                    performace[INT_QLEARNING, ind] += q_learning_reward

    performace[:3, :] /= episodes * runs
    performace[3:, :] /= 100 * runs
    labels = ['Asymptotic Sarsa', 'Asymptotic Expected Sarsa', 'Asymptotic Q-Learning',
              'Interim Sarsa', 'Interim Expected Sarsa', 'Interim Q-Learning']

    for method, label in zip(methods, labels):
        plt.plot(step_sizes, performace[method, :], label=label)
    plt.xlabel('alpha')
    plt.ylabel('reward per episode')
    plt.legend()

    plt.savefig('figure_6_6.png')
    plt.close()

if __name__ == '__main__':
    #figure_6_4()
    figure_6_6()

  0%|                                                    | 0/10 [00:00<?, ?it/s]

-2846.0 0
-118.0 0
-845.0 0
-199.0 0
-212.0 0
-277.0 0
-55.0 0
-216.0 0
-101.0 0
-723.0 0
-100.0 0
-343.0 0
-134.0 0
-74.0 0
-146.0 0
-35.0 0
-211.0 0
-101.0 0
-119.0 0
-176.0 0
-93.0 0
-89.0 0
-276.0 0
-63.0 0
-123.0 0
-99.0 0
-66.0 0
-236.0 0
-74.0 0
-71.0 0
-84.0 0
-485.0 0
-61.0 0
-96.0 0
-167.0 0
-84.0 0
-48.0 0
-174.0 0
-66.0 0
-80.0 0
-170.0 0
-418.0 0
-273.0 0
-151.0 0
-185.0 0
-202.0 0
-69.0 0
-47.0 0
-37.0 0
-36.0 0
-180.0 0
-37.0 0
-31.0 0
-66.0 0
-161.0 0
-45.0 0
-71.0 0
-27.0 0
-41.0 0
-45.0 0
-41.0 0
-36.0 0
-34.0 0
-33.0 0
-50.0 0
-49.0 0
-23.0 0
-29.0 0
-59.0 0
-253.0 0
-142.0 0
-34.0 0
-38.0 0
-53.0 0
-31.0 0
-149.0 0
-32.0 0
-25.0 0
-41.0 0
-66.0 0
-27.0 0
-25.0 0
-24.0 0
-23.0 0
-46.0 0
-148.0 0
-41.0 0
-43.0 0
-41.0 0
-159.0 0
-33.0 0
-27.0 0
-143.0 0
-50.0 0
-31.0 0
-33.0 0
-31.0 0
-136.0 0
-33.0 0
-27.0 0
-42.0 0
-132.0 0
-36.0 0
-33.0 0
-131.0 0
-25.0 0
-25.0 0
-42.0 0
-28.0 0
-35.0 0
-159.0 0
-39.0 0
-25.0 0
-141.0 0
-32.0 0
-29.0 0
-23.0 0
-47.0 0
-35.0 0
-28.0

 10%|████▍                                       | 1/10 [00:03<00:28,  3.17s/it]

 0
-19.0 0
-15.0 0
-15.0 0
-15.0 0
-17.0 0
-15.0 0
-19.0 0
-123.0 0
-22.0 0
-17.0 0
-15.0 0
-15.0 0
-15.0 0
-17.0 0
-15.0 0
-17.0 0
-21.0 0
-17.0 0
-21.0 0
-17.0 0
-17.0 0
-17.0 0
-15.0 0
-17.0 0
-119.0 0
-19.0 0
-17.0 0
-15.0 0
-17.0 0
-17.0 0
-21.0 0
-24.0 0
-15.0 0
-18.0 0
-17.0 0
-18.0 0
-19.0 0
-17.0 0
-17.0 0
-19.0 0
-118.0 0
-17.0 0
-19.0 0
-118.0 0
-25.0 0
-20.0 0
-19.0 0
-23.0 0
-18.0 0
-17.0 0
-20.0 0
-25.0 0
-30.0 0
-18.0 0
-17.0 0
-17.0 0
-20.0 0
-23.0 0
-18.0 0
-18.0 0
-2230.0 1
-497.0 1
-168.0 1
-181.0 1
-272.0 1
-412.0 1
-150.0 1
-103.0 1
-47.0 1
-178.0 1
-100.0 1
-143.0 1
-525.0 1
-151.0 1
-108.0 1
-63.0 1
-34.0 1
-190.0 1
-204.0 1
-111.0 1
-69.0 1
-21.0 1
-157.0 1
-154.0 1
-65.0 1
-34.0 1
-198.0 1
-63.0 1
-29.0 1
-185.0 1
-50.0 1
-35.0 1
-53.0 1
-54.0 1
-29.0 1
-52.0 1
-33.0 1
-56.0 1
-41.0 1
-21.0 1
-41.0 1
-48.0 1
-38.0 1
-35.0 1
-37.0 1
-46.0 1
-135.0 1
-27.0 1
-47.0 1
-300.0 1
-19.0 1
-43.0 1
-122.0 1
-28.0 1
-44.0 1
-43.0 1
-157.0 1
-30.0 1
-43.0 1
-32.0 1
-41.0 1

 20%|████████▊                                   | 2/10 [00:05<00:22,  2.85s/it]

...

 40%|█████████████████▌                          | 4/10 [00:10<00:15,  2.58s/it]

 3
-19.0 3
-17.0 3
-18.0 3
-20.0 3
-19.0 3
-17.0 3
-19.0 3
-18.0 3
-19.0 3
-121.0 3
-17.0 3
-20.0 3
-20.0 3
-19.0 3
-17.0 3
-17.0 3
-23.0 3
-19.0 3
-30.0 3
-19.0 3
-17.0 3
-19.0 3
-17.0 3
-117.0 3
-19.0 3
-17.0 3
-118.0 3
-20.0 3
-19.0 3
-18.0 3
-17.0 3
-24.0 3
-17.0 3
-121.0 3
-17.0 3
-17.0 3
-24.0 3
-21.0 3
-21.0 3
-18.0 3
-21.0 3
-24.0 3
-20.0 3
-18.0 3
-17.0 3
-19.0 3
-17.0 3
-20.0 3
-17.0 3
-17.0 3
-17.0 3
-20.0 3
-21.0 3
-19.0 3
-17.0 3
-17.0 3
-19.0 3
-20.0 3
-17.0 3
-19.0 3
-17.0 3
-21.0 3
-21.0 3
-22.0 3
-19.0 3
-17.0 3
-17.0 3
-25.0 3
-18.0 3
-24.0 3
-19.0 3
-22.0 3
-17.0 3
-21.0 3
-17.0 3
-18.0 3
-20.0 3
-19.0 3
-19.0 3
-17.0 3
-19.0 3
-2033.0 4
-397.0 4
-83.0 4
-65.0 4
-59.0 4
-225.0 4
-178.0 4
-25.0 4
-59.0 4
-62.0 4
-31.0 4
-137.0 4
-68.0 4
-37.0 4
-70.0 4
-30.0 4
-138.0 4
-29.0 4
-173.0 4
-58.0 4
-38.0 4
-35.0 4
-47.0 4
-51.0 4
-34.0 4
-33.0 4
-240.0 4
-31.0 4
-19.0 4
-23.0 4
-45.0 4
-21.0 4
-42.0 4
-41.0 4
-23.0 4
-154.0 4
-29.0 4
-35.0 4
-24.0 4
-21.0 4
-124.0 4
-31.0 

 40%|█████████████████▌                          | 4/10 [00:11<00:17,  2.92s/it]

 4
-18.0 4
-95.0 4
-37.0 4
-23.0 4
-30.0 4
-25.0 4
-25.0 4
-19.0 4
-23.0 4
-127.0 4
-23.0 4


KeyboardInterrupt: 